# SIMP example: Point Load Cantilever

## Commented Program

What follows is a program spliced with comments.

In [1]:
using TopOpt

### Define the problem

In [2]:
E = 1.0 # Young’s modulus
v = 0.3 # Poisson’s ratio
f = 1.0; # downward force

nels = (30, 10, 10)
problem = PointLoadCantilever(Val{:Linear}, nels, (1.0, 1.0, 1.0), E, v, f);

See also the detailed API of `PointLoadCantilever`:

### Parameter settings

In [3]:
V = 0.3 # volume fraction
xmin = 1e-6 # minimum density
rmin = 2.0; # density filter radius

### Define a finite element solver

In [4]:
penalty = TopOpt.PowerPenalty(3.0)
solver = FEASolver(Direct, problem; xmin=xmin, penalty=penalty)

### Define compliance objective

In [5]:
comp = TopOpt.Compliance(problem, solver)
filter = DensityFilter(solver; rmin=rmin)
obj = x -> comp(filter(x))

#1 (generic function with 1 method)

### Define volume constraint

In [6]:
volfrac = TopOpt.Volume(problem, solver)
constr = x -> volfrac(filter(x)) - V

#3 (generic function with 1 method)

You can enable the iteration printouts with `Nonconvex.NonconvexCore.show_residuals[] = true`

### Define subproblem optimizer

In [7]:
mma_options =
    options = MMAOptions(;
        maxiter=3000, tol=Nonconvex.Tolerance(; x=1e-3, f=1e-3, kkt=0.001)
    )
convcriteria = Nonconvex.KKTCriteria()
x0 = fill(V, length(solver.vars))
optimizer = Optimizer(
    obj, constr, x0, MMA87(); options=mma_options, convcriteria=convcriteria
)

Internal error: encountered unexpected error in runtime:
TypeError(func=:<:, context="", expected=Type, got=TApprox<:NonconvexMMA.AbstractMMAApprox)
jl_type_error_rt at /buildworker/worker/package_linux64/build/src/rtutils.c:119
jl_type_error at /buildworker/worker/package_linux64/build/src/rtutils.c:127
jl_f_issubtype at /buildworker/worker/package_linux64/build/src/builtins.c:491
apply_type_nothrow at ./compiler/tfuncs.jl:1245
_builtin_nothrow at ./compiler/tfuncs.jl:1566
stmt_effect_free at ./compiler/ssair/queries.jl:36
check_effect_free! at ./compiler/ssair/inlining.jl:1128 [inlined]
process_simple! at ./compiler/ssair/inlining.jl:1185
assemble_inline_todo! at ./compiler/ssair/inlining.jl:1358
ssa_inlining_pass! at ./compiler/ssair/inlining.jl:80
jfptr_ssa_inlining_passNOT._12916.clone_1 at /opt/hostedtoolcache/julia/1.7.2/x64/lib/julia/sys.so (unknown line)
_jl_invoke at /buildworker/worker/package_linux64/build/src/gf.c:2247 [inlined]
jl_apply_generic at /buildworker/worker/pack

### Define SIMP optimizer

In [8]:
simp = SIMP(optimizer, solver, penalty.p);

### Solve

In [9]:
result = simp(x0);

@show result.convstate
@show optimizer.workspace.iter
@show result.objval

result.convstate = NonconvexCore.ConvergenceState{Float64}
  Δx: Float64 0.038385266065596335
  Δf: Float64 0.0002795828717765403
  relΔf: Float64 6.6190491590805865e-6
  kkt_residual: Float64 0.0009088013654401719
  ipopt_residual: Float64 0.0009088013654401719
  infeas: Float64 0.0
  x_converged: Bool false
  fabs_converged: Bool true
  frel_converged: Bool true
  kkt_converged: Bool true
  ipopt_converged: Bool true
  infeas_converged: Bool true
  f_increased: Bool false
  converged: Bool true

optimizer.workspace.iter = 45
result.objval = 42.238129073846544


42.238129073846544

### (Optional) Visualize the result using Makie.jl
Need to run `using Pkg; Pkg.add(Makie)` first
```julia
using TopOpt.TopOptProblems.Visualization: visualize
fig = visualize(problem; topology = result.topology,
    problem; topology = result.topology, default_exagg_scale = 0.07,
    scale_range = 10.0, vector_linewidth = 3, vector_arrowsize = 0.5,
)
Makie.display(fig)
```

or convert it to a Mesh
Need to run `using Pkg; Pkg.add(GeometryBasics)` first
```julia
import Makie, GeometryBasics
result_mesh = GeometryBasics.Mesh(problem, result.topology);
Makie.mesh(result_mesh)
```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*